In [5]:
from bs4 import BeautifulSoup
import requests
import re
import json
import os

In [6]:
url = "https://www.doktortakvimi.com/ara?q=Kad%C4%B1n%20Hastal%C4%B1klar%C4%B1%20Ve%20Do%C4%9Fum&loc=Ankara&filters%5Bspecializations%5D%5B0%5D=56&page=1"

In [7]:
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")


In [ ]:
soup